In [48]:
import boto3 as boto


In [49]:
s3 = boto.resource('s3', aws_access_key_id='redacting_for_security', aws_secret_access_key='redacting_for_security')

In [50]:
try: 
    s3.create_bucket(Bucket='hw3-nosql', CreateBucketConfiguration={'LocationConstraint': 'us-west-2'}) 
except Exception as e: 
    print (e)

In [51]:
bucket = s3.Bucket('hw3-nosql') 

In [52]:
bucket.Acl().put(ACL='public-read') 

{'ResponseMetadata': {'RequestId': 'NABW2JFE2J6D275S',
  'HostId': '1fzEdJ7PmNiqNJQqyZgNb95ErBuZcDCVqtNlxZfOW3K+PbPp8NnZPqzMpPEbGag6GV+aMZI3U9I=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '1fzEdJ7PmNiqNJQqyZgNb95ErBuZcDCVqtNlxZfOW3K+PbPp8NnZPqzMpPEbGag6GV+aMZI3U9I=',
   'x-amz-request-id': 'NABW2JFE2J6D275S',
   'date': 'Wed, 20 Oct 2021 03:11:49 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [53]:
body = open('.\exp1.csv', 'rb') 

In [54]:
o = s3.Object('hw3-nosql', 'test').put(Body=body)

In [55]:
s3.Object('hw3-nosql', 'test').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'DAD09QB50N3NW3DN',
  'HostId': 'jdJ4OFe2rI2SOrLIqEY5csqXFMTM3+ynPU3oKbPM9lHQSR8NJ0troG0hUCJJizC58KNqm3Ju2CM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'jdJ4OFe2rI2SOrLIqEY5csqXFMTM3+ynPU3oKbPM9lHQSR8NJ0troG0hUCJJizC58KNqm3Ju2CM=',
   'x-amz-request-id': 'DAD09QB50N3NW3DN',
   'date': 'Wed, 20 Oct 2021 03:11:56 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [56]:
dyndb = boto.resource('dynamodb', region_name='us-west-2', aws_access_key_id='redacting_for_security', aws_secret_access_key='redacting_for_security' )

In [57]:
try: 
    table = dyndb.create_table( 
        TableName='DataTable', 
        KeySchema=[ 
            { 
                'AttributeName': 'PartitionKey', 
                'KeyType': 'HASH' 
            }, 
            { 
                'AttributeName': 'RowKey', 
                'KeyType': 'RANGE' 
            } 
        ], 
        AttributeDefinitions=[ 
            { 
                'AttributeName': 'PartitionKey', 
                'AttributeType': 'S' 
            }, 
            { 
                'AttributeName': 'RowKey', 
                'AttributeType': 'S' 
            }, 
 
        ], 
        ProvisionedThroughput={ 
            'ReadCapacityUnits': 5, 
            'WriteCapacityUnits': 5 
        } 
    ) 
except Exception as e: 
    print (e) 
    #if there is an exception, the table may already exist.   if so... 
    table = dyndb.Table("DataTable")

An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: DataTable


In [58]:
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

In [59]:
print(table.item_count)

0


In [60]:
import csv 

In [61]:
with open('.\experiments.csv', 'r') as csvfile: 
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvf)
    for item in csvf: 
        print(item)
        body = open('.\\'+item[3] + '.csv', 'rb') 
        s3.Object('hw3-nosql', item[3]).put(Body=body ) 
        md = s3.Object('hw3-nosql', item[3]).Acl().put(ACL='public-read') 
         
        url = " https://s3-us-west-2.amazonaws.com/datacont-name/"+item[3] 
        metadata_item = {'PartitionKey': item[0], 'RowKey': item[1],  
                 'description' : item[4], 'date' : item[2], 'url':url}  
        try: 
            table.put_item(Item=metadata_item) 
        except: 
            print("item may already be there or another failure")

['experiment1', '1', '3/15/2002', 'exp1', 'this is the comment ']
['experiment1', '2', '3/15/2002', 'exp2', 'this is the comment2 ']
['experiment2', '3', '3/16/2002', 'exp3', 'this is the comment3 ']
['experiment3', '4', '3/16/2002', 'exp4', 'this is the comment233']


In [62]:
response = table.get_item( 
    Key={ 
        'PartitionKey': 'experiment3', 
        'RowKey': '4' 
    } 
) 
item = response['Item'] 
print(item)

{'PartitionKey': 'experiment3', 'RowKey': '4', 'date': '3/16/2002', 'description': 'this is the comment233', 'url': ' https://s3-us-west-2.amazonaws.com/datacont-name/exp4'}


In [63]:
response

{'Item': {'PartitionKey': 'experiment3',
  'RowKey': '4',
  'date': '3/16/2002',
  'description': 'this is the comment233',
  'url': ' https://s3-us-west-2.amazonaws.com/datacont-name/exp4'},
 'ResponseMetadata': {'RequestId': 'JETFF3UNBR5G9VG4K42HUP0L97VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 20 Oct 2021 03:12:39 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '203',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'JETFF3UNBR5G9VG4K42HUP0L97VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3668776648'},
  'RetryAttempts': 0}}